# Data Exploration

## Importing libraries and data

In [14]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("imigration-exploration-spark").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/26 13:39:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/26 13:39:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df_i = spark.read.parquet('/Users/felipeaugustonogueira/Documents/personal/imigration_exploration/imigration_parquet')
df_i.createOrReplaceTempView("imigration")

23/03/26 13:40:31 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Total number of rows on the DataFrame.

In [24]:
num_rows = df_i.count()
num_rows

3096313

In [15]:
spark.sql("""
SELECT
    *
FROM imigration
LIMIT 5
"""
).toPandas().head(4)

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94bir,i94visa,count,dtadfile,visapost,occup,entdepa,entdepd,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,5748517.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,CA,20582.0,40.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1976.0,10292016,F,None,QF,9.495387e+10,00011,B1
1,5748518.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,NV,20591.0,32.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1984.0,10292016,F,None,VA,9.495562e+10,00007,B1
2,5748519.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20582.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,M,None,DL,9.495641e+10,00040,B1
3,5748520.0,2016.0,4.0,245.0,438.0,LOS,20574.0,1.0,WA,20588.0,29.0,1.0,1.0,20160430,SYD,None,G,O,None,M,1987.0,10292016,F,None,DL,9.495645e+10,00040,B1


## Data Exploration
Which is the windows of time considered in the `i94yr` and `i94mon`?<br>
_Very small period of april, 2016._

In [10]:
spark.sql("""
SELECT DISTINCT
    i94yr
FROM imigration
""").toPandas().head()

,i94yr
0,2016.0


In [11]:
spark.sql("""
SELECT DISTINCT
    i94mon
FROM imigration
""").toPandas().head()

,i94mon
0,4.0


Is the column `cicid` a good candidate for an imigrant id?<br>
_There are no duplicates on this column. So it is ok to use it as an id column._

In [12]:
spark.sql("""
SELECT
    cicid,
    COUNT(cicid) AS count_cicid
FROM imigration
GROUP BY 1
ORDER BY 2 DESC
""").toPandas().head()

,cicid,count_cicid
0,5748877.0,1
1,5749231.0,1
2,5749277.0,1
3,5750090.0,1
4,5750830.0,1


How many occupations type do we have in this dataset?<br>
_There are only 5. The number of rows with this information does not represent even 1% of the dataset. So, it can not be used to retrieve any significant information._

In [16]:
spark.sql("""
SELECT
    occup,
    COUNT(occup) AS count_occup
FROM imigration
GROUP BY 1
ORDER BY 2 DESC
""").toPandas().head()

,occup,count_occup
0,STU,4719
1,OTH,661
2,NRR,345
3,MKT,280
4,EXA,196


In [17]:
spark.sql("""
WITH occup AS (
SELECT
    occup,
    COUNT(occup) AS count_occup
FROM imigration
GROUP BY 1
ORDER BY 2 DESC
)
SELECT
    SUM(count_occup) AS num_registers
FROM occup
""").toPandas().head()

,num_registers
0,8126


In [28]:
(8126/num_rows)*100

0.2624411679310199

In the imigration dataset we have the column that refers to the port itself (`i94port`) and one refering to the transportation modal (`i94mode`). Do we have cases of one `i94port` id having more than one type of transportation modal?<br>
_By what we can see below we can have people arriving at the same place by more than one type of transportation modal. So, when performing joins with the airport dataset, we have to be cautious to not attribute an airport to an imigrant when he could have arrived by another transportation modal. In this case, we have to perform the JOIN operation using two columns `i94port` and `i94mode`._

In [29]:
spark.sql("""
WITH counting_modes AS (
SELECT DISTINCT
    i94port,
    i94mode
FROM imigration
)
SELECT
    i94port,
    COUNT(i94mode) AS count_port_modals
FROM counting_modes
GROUP BY 1
ORDER BY 2 DESC
""").toPandas().head()

,i94port,count_port_modals
0,FMY,4
1,SPM,4
2,DET,4
3,AGA,4
4,SFR,4


In [30]:
spark.sql("""
SELECT DISTINCT
    i94port,
    i94mode
FROM imigration
WHERE i94port = 'SFR'
""").toPandas().head()

,i94port,i94mode
0,SFR,NaN
1,SFR,2.0
2,SFR,3.0
3,SFR,9.0
4,SFR,1.0


Which are the most common ports of arrival for aerial mode?<br>
_There is a little problem here. Since we are looking at arrivals at US it does not make much sense to see an airport from Lagos, Nigeria in the list. This tells us that the list in `i94port` on imigrant dataset does not seems to talk to the `iata_codes` at the airports codes dataset. We will have to adjust that by hand. Look at the ports id and adjust it by hand. For example: LOS as Los Angeles and NYC as New York._

In [32]:
# Ports with most arrivals
spark.sql("""
SELECT
    i94port,
    SUM(count) AS most_arrivals_at
FROM imigration
WHERE i94mode = 1
GROUP BY 1
ORDER BY 2 DESC
""").toPandas().head(10)

,i94port,most_arrivals_at
0,NYC,484299.0
1,MIA,340361.0
2,LOS,308939.0
3,SFR,151904.0
4,ORL,147752.0
5,NEW,135636.0
6,HHW,135565.0
7,CHI,129771.0
8,HOU,99861.0
9,FTL,93842.0


In [38]:
df_a = spark.read.option("header","true").csv('/Users/felipeaugustonogueira/Documents/personal/imigration_exploration/airport-codes_csv.csv')
df_a.createOrReplaceTempView("airports")

In [39]:
spark.sql("""
SELECT
    *
FROM airports LIMIT 5
""").toPandas().head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11,NA,US,US-PA,Bensalem,00A,None,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435,NA,US,US-KS,Leoti,00AA,None,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450,NA,US,US-AK,Anchor Point,00AK,None,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820,NA,US,US-AL,Harvest,00AL,None,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237,NA,US,US-AR,Newport,None,None,None,"-91.254898, 35.6087"


In [42]:
# Ports with most arrivals on the airports codes dataset.
spark.sql("""
SELECT
    type,
    name,
    municipality,
    iso_country,
    iata_code, 
    local_code,
    ident
FROM airports
WHERE iata_code IN ('NYC', 'MIA', 'LOS', 'SFR', 'ORL', 'NEW', 'HHW', 'CHI', 'HOU', 'FTL')
""").toPandas().head()

,type,name,municipality,iso_country,iata_code,local_code,ident
0,large_airport,Murtala Muhammed International Airport,Lagos,NG,LOS,None,DNMM
1,large_airport,William P Hobby Airport,Houston,US,HOU,HOU,KHOU
2,large_airport,Miami International Airport,Miami,US,MIA,MIA,KMIA
3,medium_airport,Lakefront Airport,New Orleans,US,NEW,NEW,KNEW
4,small_airport,Orlando Executive Airport,Orlando,US,ORL,ORL,KORL


In [51]:
spark.sql("""
SELECT
    i94port,
    COUNT(i94port) AS count_arrivals,
    (COUNT(i94port)/3096313)*100 AS percent
FROM imigration
GROUP BY 1
ORDER BY 2 DESC
""").toPandas().head(30)

,i94port,count_arrivals,percent
0,NYC,485916,15.693375
1,MIA,343941,11.108082
2,LOS,310163,10.017172
3,SFR,152586,4.927990
4,ORL,149195,4.818473
5,HHW,142720,4.609353
6,NEW,136122,4.396261
7,CHI,130564,4.216757
8,HOU,101481,3.277479
9,FTL,95977,3.099719


In [54]:
df_sas = spark.read \
    .option("header","true") \
    .option("delimiter", "|") \
    .csv('/Users/felipeaugustonogueira/Documents/personal/imigration_exploration/sas_descriptive_information.csv')
df_sas.createOrReplaceTempView("sas")


Generating the name of the city of the ports so we can correct the id when getting the airports names.<br>
_Looking at the query below we can see that there are some `iata_code` that does not exist (New York, for example) and some others that return the wrong place. Unfortunately, we have no way of knowing from which airport does the imigrant arrives in case we have multiple airports in the city (New York, for example). In this case, the best thing we can do is to discart the airport database and use the information we retrieved from the SAS data base._

Above we can see the Los Angeles Airport wrongly defined as Murtala Muhammed International Airport, because the i94port for it is LOS. But, in this case the iata code and the i94 information does not match. If it did, the i94port would probrably be LAX. We did tried to adjust the information, but there is no way with what we have so far.

In [61]:
spark.sql("""
SELECT
    i.i94port,
    s.description AS port_city,
    a.iata_code,
    a.iso_country,
    a.name,
    COUNT(i.i94port) AS count_arrivals,
    ROUND((COUNT(i.i94port)/3096313)*100,2) AS percent
FROM imigration i
LEFT JOIN sas s ON s.id = i.i94port AND s.column = 'i94port'
LEFT JOIN airports a ON a.iata_code = i.i94port
GROUP BY 1, 2, 3, 4, 5
ORDER BY 6 DESC
""").toPandas().head(20)

,i94port,port_city,iata_code,iso_country,name,count_arrivals,percent
0,NYC,"NEW YORK, NY",None,None,None,485916,15.69
1,MIA,"MIAMI, FL",MIA,US,Miami International Airport,343941,11.11
2,LOS,"LOS ANGELES, CA",LOS,NG,Murtala Muhammed International Airport,310163,10.02
3,SFR,"SAN FRANCISCO, CA",SFR,US,San Fernando Airport,152586,4.93
4,ORL,"ORLANDO, FL",ORL,US,Orlando Executive Airport,149195,4.82
5,HHW,"HONOLULU, HI",None,None,None,142720,4.61
6,NEW,"NEWARK/TETERBORO, NJ",NEW,US,Lakefront Airport,136122,4.40
7,CHI,"CHICAGO, IL",None,None,None,130564,4.22
8,HOU,"HOUSTON, TX",HOU,US,William P Hobby Airport,101481,3.28
9,FTL,"FORT LAUDERDALE, FL",None,None,None,95977,3.10


In [65]:
spark.sql("""
WITH adj_query AS (
    SELECT
        i.i94port,
        s.description AS port_city,
        a.iata_code,
        a.iso_country,
        a.name,
        COUNT(i.i94port) AS count_arrivals,
        ROUND((COUNT(i.i94port)/3096313)*100,2) AS percent
    FROM imigration i
    LEFT JOIN sas s ON s.id = i.i94port AND s.column = 'i94port'
    LEFT JOIN airports a ON a.iata_code = i.i94port
    GROUP BY 1, 2, 3, 4, 5
    ORDER BY 6 DESC LIMIT 20
)
SELECT
    SUM(percent)
FROM adj_query
""").toPandas().head()

,sum(percent)
0,86.1


In [75]:
spark.sql("""
WITH adj_query AS (
    SELECT
        i.i94port,
        i.i94mode,
        s.description AS port_city,
        a.iata_code,
        a.iso_country,
        a.name,
        COUNT(i.i94port) AS count_arrivals,
        ROUND((COUNT(i.i94port)/3096313)*100,2) AS percent
    FROM imigration i
    LEFT JOIN sas s ON s.id = i.i94port AND s.column = 'i94port'
    LEFT JOIN airports a ON a.iata_code = i.i94port
    WHERE i.i94mode = 1
    GROUP BY 1, 2, 3, 4, 5, 6
    ORDER BY 7 DESC LIMIT 20
),
imig_base_city AS (
SELECT DISTINCT
    i94port,
    LOWER(SPLIT(port_city,',')[0]) AS city
FROM adj_query
), final AS (
SELECT
    i.i94port,
    i.city,
    LOWER(a.municipality) AS municipality,
    a.iata_code
FROM imig_base_city i
LEFT JOIN airports a ON LOWER(municipality) = i.city
)
SELECT
    * 
FROM final
""").toPandas().head(100)

,i94port,city,municipality,iata_code
0,NYC,new york,new york,None
1,NYC,new york,new york,None
2,NYC,new york,new york,None
3,NYC,new york,new york,None
4,NYC,new york,new york,LGA
5,NYC,new york,new york,JFK
6,NYC,new york,new york,JRB
7,NYC,new york,new york,JRA
8,NYC,new york,new york,JPB
9,NYC,new york,new york,QNY


Above, we have multiple ports in New York city. How do we know from which the imigrant arrived in the US?<br>
If there is no way of knowing, and the only information we can retrieve is the city, it would be better to get the city information from the SAS .csv file where we can have the city for every i94port id.

In [77]:
spark.sql("""
SELECT
    *
FROM airports
WHERE municipality = 'New York'
""").toPandas().head(20)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,6N5,heliport,East 34th Street Heliport,10,NA,US,US-NY,New York,6N5,TSS,6N5,"-73.97209930419922, 40.74259948730469"
1,6N6,seaplane_base,Evers Seaplane Base,None,NA,US,US-NY,New York,6N6,None,6N6,"-73.81620025634766, 40.84590148925781"
2,6N7,seaplane_base,New York Skyports Inc Seaplane Base,None,NA,US,US-NY,New York,None,QNY,6N7,"-73.9729, 40.734001"
3,JPB,closed,Pan Am Building Heliport,870,NA,US,US-NY,New York,None,JPB,None,"-73.9765, 40.7533"
4,JRA,heliport,West 30th St. Heliport,7,NA,US,US-NY,New York,KJRA,JRA,JRA,"-74.007103, 40.754501"
5,JRB,heliport,Downtown-Manhattan/Wall St Heliport,7,NA,US,US-NY,New York,KJRB,JRB,JRB,"-74.00900269, 40.70119858"
6,KJFK,large_airport,John F Kennedy International Airport,13,NA,US,US-NY,New York,KJFK,JFK,JFK,"-73.77890015, 40.63980103"
7,KLGA,large_airport,La Guardia Airport,21,NA,US,US-NY,New York,KLGA,LGA,LGA,"-73.87259674, 40.77719879"
8,KNOP,closed,Floyd Bennett Field,16,NA,US,US-NY,New York,None,None,None,"-73.890999, 40.591"
9,NK39,heliport,One Police Plaza Heliport,244,NA,US,US-NY,New York,NK39,None,NK39,"-73.99960327148438, 40.71260070800781"
